In [1]:
import numpy as np
import pandas as pd
import acquire as a 
import prepare as p
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

# Zillow

For the following, iterate through the steps you would take to create functions: Write the code to do the following in a jupyter notebook, test it, convert to functions, then create the file to house those functions.

You will have a zillow.ipynb file and a helper file for each section in the pipeline.

### acquire & summarize

1. Acquire data from mySQL using the python module to connect and query. You will want to end with a single dataframe. Make sure to include: the logerror, all fields related to the properties that are available. You will end up using all the tables in the database.

- Be sure to do the correct join (inner, outer, etc.). We do not want to eliminate properties purely because they may have a null value for airconditioningtypeid.
- Only include properties with a transaction in 2017, and include only the last transaction for each property (so no duplicate property ID's), along with zestimate error and date of transaction.
- Only include properties that include a latitude and longitude value.

In [2]:
#write a query (this is from sql)

In [3]:
sql_query = '''
SELECT prop.parcelid,  prop.basementsqft, bathroomcnt, bedroomcnt, decktypeid, calculatedfinishedsquarefeet,
fips, fireplacecnt, garagecarcnt, hashottuborspa, latitude, longitude, lotsizesquarefeet, poolcnt,
yearbuilt, numberofstories, prop.airconditioningtypeid, airconditioningdesc, prop.architecturalstyletypeid,
architecturalstyledesc, prop.buildingclasstypeid, buildingclassdesc, prop.heatingorsystemtypeid,
heatingorsystemdesc, prop.storytypeid, storydesc, prop.propertylandusetypeid, propertylandusedesc, 
prop.typeconstructiontypeid, typeconstructiondesc, unitcnt, taxvaluedollarcnt, taxamount, logerror, transactiondate 
from properties_2017 as prop
join predictions_2017 as pred USING (parcelid)
LEFT JOIN airconditioningtype USING (airconditioningtypeid)
LEFT JOIN architecturalstyletype USING (architecturalstyletypeid)
LEFT JOIN buildingclasstype USING (buildingclasstypeid)
LEFT JOIN heatingorsystemtype USING (heatingorsystemtypeid)
LEFT JOIN propertylandusetype USING (propertylandusetypeid)
LEFT JOIN storytype USING(storytypeid)
LEFT JOIN typeconstructiontype USING (typeconstructiontypeid)
WHERE transactiondate like '2017%' 
AND latitude >= 0 AND longitude <= 0;
'''

In [4]:
sql_query2 ='''
SELECT prop.parcelid,  prop.basementsqft, bathroomcnt, bedroomcnt, decktypeid, calculatedfinishedsquarefeet,
fips, fireplacecnt, garagecarcnt, hashottuborspa, latitude, longitude, lotsizesquarefeet, poolcnt,
yearbuilt, numberofstories, prop.airconditioningtypeid, airconditioningdesc, prop.architecturalstyletypeid,
architecturalstyledesc, prop.buildingclasstypeid, buildingclassdesc, prop.heatingorsystemtypeid,
heatingorsystemdesc, prop.storytypeid, storydesc, prop.propertylandusetypeid, propertylandusedesc, 
prop.typeconstructiontypeid, typeconstructiondesc, unitcnt, taxvaluedollarcnt, taxamount, logerror, transactiondate 
from properties_2017 as prop
join predictions_2017 as pred USING (parcelid)
LEFT JOIN airconditioningtype USING (airconditioningtypeid)
LEFT JOIN architecturalstyletype USING (architecturalstyletypeid)
LEFT JOIN buildingclasstype USING (buildingclasstypeid)
LEFT JOIN heatingorsystemtype USING (heatingorsystemtypeid)
LEFT JOIN propertylandusetype USING (propertylandusetypeid)
LEFT JOIN storytype USING(storytypeid)
LEFT JOIN typeconstructiontype USING (typeconstructiontypeid)
WHERE transactiondate like '2017%'  
AND latitude != 'NULL' AND longitude != 'NULL';
'''

In [5]:
#acquire my df
df= a.get_data_from_sql('zillow',sql_query2)

In [6]:
df.head()

,parcelid,basementsqft,bathroomcnt,bedroomcnt,decktypeid,calculatedfinishedsquarefeet,fips,fireplacecnt,garagecarcnt,hashottuborspa,...,storydesc,propertylandusetypeid,propertylandusedesc,typeconstructiontypeid,typeconstructiondesc,unitcnt,taxvaluedollarcnt,taxamount,logerror,transactiondate
0,14297519,NaN,3.5,4.0,NaN,3100.0,6059.0,NaN,2.0,NaN,...,None,261.0,Single Family Residential,NaN,None,NaN,1023282.0,11013.72,0.025595,2017-01-01
1,17052889,NaN,1.0,2.0,NaN,1465.0,6111.0,1.0,1.0,NaN,...,None,261.0,Single Family Residential,NaN,None,NaN,464000.0,5672.48,0.055619,2017-01-01
2,14186244,NaN,2.0,3.0,NaN,1243.0,6059.0,NaN,2.0,NaN,...,None,261.0,Single Family Residential,NaN,None,NaN,564778.0,6488.30,0.005383,2017-01-01
3,12177905,NaN,3.0,4.0,NaN,2376.0,6037.0,NaN,NaN,NaN,...,None,261.0,Single Family Residential,NaN,None,1.0,145143.0,1777.51,-0.103410,2017-01-01
4,10887214,NaN,3.0,3.0,NaN,1312.0,6037.0,NaN,NaN,NaN,...,None,266.0,Condominium,NaN,None,1.0,119407.0,1533.89,0.006940,2017-01-01


In [7]:
df.shape

(77579, 35)

In [8]:
#checking the unique values
df['parcelid'].nunique()

77381

In [9]:
#getting the latest transaction . fist we need to sort the transaction dates and then drop duplicates and keep last
df1 = df.sort_values(by ='transactiondate', ascending=True).drop_duplicates( subset = 'parcelid' ,keep= 'last')

In [10]:
#just checking if the df1 has the latest transactiondate
df1[df1['parcelid'] == 10857130]

,parcelid,basementsqft,bathroomcnt,bedroomcnt,decktypeid,calculatedfinishedsquarefeet,fips,fireplacecnt,garagecarcnt,hashottuborspa,...,storydesc,propertylandusetypeid,propertylandusedesc,typeconstructiontypeid,typeconstructiondesc,unitcnt,taxvaluedollarcnt,taxamount,logerror,transactiondate
44944,10857130,NaN,0.0,0.0,NaN,NaN,6037.0,NaN,NaN,NaN,...,None,260.0,Residential General,NaN,None,NaN,1523182.0,18177.58,0.290908,2017-08-25


In [11]:
#we have the same number as nunique
df1.shape

(77381, 35)

In [12]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77381 entries, 0 to 77578
Data columns (total 35 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   parcelid                      77381 non-null  int64  
 1   basementsqft                  50 non-null     float64
 2   bathroomcnt                   77381 non-null  float64
 3   bedroomcnt                    77381 non-null  float64
 4   decktypeid                    614 non-null    float64
 5   calculatedfinishedsquarefeet  77185 non-null  float64
 6   fips                          77381 non-null  float64
 7   fireplacecnt                  8276 non-null   float64
 8   garagecarcnt                  25474 non-null  float64
 9   hashottuborspa                1538 non-null   float64
 10  latitude                      77381 non-null  float64
 11  longitude                     77381 non-null  float64
 12  lotsizesquarefeet             69142 non-null  float64
 13  p

**2. Summarize your data (summary stats, info, dtypes, shape, distributions, value_counts, etc.)**

In [13]:
df1.head(3)

,parcelid,basementsqft,bathroomcnt,bedroomcnt,decktypeid,calculatedfinishedsquarefeet,fips,fireplacecnt,garagecarcnt,hashottuborspa,...,storydesc,propertylandusetypeid,propertylandusedesc,typeconstructiontypeid,typeconstructiondesc,unitcnt,taxvaluedollarcnt,taxamount,logerror,transactiondate
0,14297519,NaN,3.5,4.0,NaN,3100.0,6059.0,NaN,2.0,NaN,...,None,261.0,Single Family Residential,NaN,None,NaN,1023282.0,11013.72,0.025595,2017-01-01
1,17052889,NaN,1.0,2.0,NaN,1465.0,6111.0,1.0,1.0,NaN,...,None,261.0,Single Family Residential,NaN,None,NaN,464000.0,5672.48,0.055619,2017-01-01
2,14186244,NaN,2.0,3.0,NaN,1243.0,6059.0,NaN,2.0,NaN,...,None,261.0,Single Family Residential,NaN,None,NaN,564778.0,6488.30,0.005383,2017-01-01


In [14]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77381 entries, 0 to 77578
Data columns (total 35 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   parcelid                      77381 non-null  int64  
 1   basementsqft                  50 non-null     float64
 2   bathroomcnt                   77381 non-null  float64
 3   bedroomcnt                    77381 non-null  float64
 4   decktypeid                    614 non-null    float64
 5   calculatedfinishedsquarefeet  77185 non-null  float64
 6   fips                          77381 non-null  float64
 7   fireplacecnt                  8276 non-null   float64
 8   garagecarcnt                  25474 non-null  float64
 9   hashottuborspa                1538 non-null   float64
 10  latitude                      77381 non-null  float64
 11  longitude                     77381 non-null  float64
 12  lotsizesquarefeet             69142 non-null  float64
 13  p

In [15]:
df1.dtypes

parcelid                          int64
basementsqft                    float64
bathroomcnt                     float64
bedroomcnt                      float64
decktypeid                      float64
calculatedfinishedsquarefeet    float64
fips                            float64
fireplacecnt                    float64
garagecarcnt                    float64
hashottuborspa                  float64
latitude                        float64
longitude                       float64
lotsizesquarefeet               float64
poolcnt                         float64
yearbuilt                       float64
numberofstories                 float64
airconditioningtypeid           float64
airconditioningdesc              object
architecturalstyletypeid        float64
architecturalstyledesc           object
buildingclasstypeid             float64
buildingclassdesc                object
heatingorsystemtypeid           float64
heatingorsystemdesc              object
storytypeid                     float64


In [16]:
df1.describe()

,parcelid,basementsqft,bathroomcnt,bedroomcnt,decktypeid,calculatedfinishedsquarefeet,fips,fireplacecnt,garagecarcnt,hashottuborspa,...,architecturalstyletypeid,buildingclasstypeid,heatingorsystemtypeid,storytypeid,propertylandusetypeid,typeconstructiontypeid,unitcnt,taxvaluedollarcnt,taxamount,logerror
count,7.738100e+04,50.000000,77381.000000,77381.000000,614.0,77185.000000,77381.000000,8276.000000,25474.000000,1538.0,...,206.000000,15.000000,49440.000000,50.0,77381.000000,222.000000,50563.000000,7.738000e+04,77376.000000,77381.000000
mean,1.300715e+07,679.720000,2.299130,3.053489,66.0,1785.219939,6048.819232,1.191155,1.815459,1.0,...,7.388350,3.933333,3.920510,7.0,261.826340,6.040541,1.110021,4.901345e+05,5995.532346,0.016625
std,3.481346e+06,689.703546,0.996651,1.139096,0.0,954.049286,20.748199,0.491080,0.588345,0.0,...,2.734542,0.258199,3.592779,0.0,5.141199,0.557285,1.171154,6.534440e+05,7622.844153,0.170191
min,1.071186e+07,38.000000,0.000000,0.000000,66.0,128.000000,6037.000000,1.000000,0.000000,1.0,...,2.000000,3.000000,1.000000,7.0,31.000000,4.000000,1.000000,1.000000e+03,19.920000,-4.655420
25%,1.153830e+07,273.000000,2.000000,2.000000,66.0,1182.000000,6037.000000,1.000000,2.000000,1.0,...,7.000000,4.000000,2.000000,7.0,261.000000,6.000000,1.000000,2.070000e+05,2715.632500,-0.024377
50%,1.253157e+07,515.000000,2.000000,3.000000,66.0,1543.000000,6037.000000,1.000000,2.000000,1.0,...,7.000000,4.000000,2.000000,7.0,261.000000,6.000000,1.000000,3.589755e+05,4450.695000,0.006627
75%,1.421183e+07,796.500000,3.000000,4.000000,66.0,2113.000000,6059.000000,1.000000,2.000000,1.0,...,7.000000,4.000000,7.000000,7.0,266.000000,6.000000,1.000000,5.690015e+05,6927.790000,0.039203
max,1.676893e+08,3560.000000,18.000000,16.000000,66.0,35640.000000,6111.000000,5.000000,14.000000,1.0,...,21.000000,4.000000,24.000000,7.0,275.000000,13.000000,237.000000,4.906124e+07,586639.300000,5.262999


In [17]:
df1['parcelid'].nunique()

77381

In [18]:
len(df1.parcelid)

77381

In [19]:
df1.select_dtypes(include = 'O').columns.to_list()

['airconditioningdesc',
 'architecturalstyledesc',
 'buildingclassdesc',
 'heatingorsystemdesc',
 'storydesc',
 'propertylandusedesc',
 'typeconstructiondesc',
 'transactiondate']

In [20]:
num_cols = df1.select_dtypes(exclude = 'O').columns.to_list()
cat_cols = df1.select_dtypes(include = 'O').columns.to_list()
for col in df.columns:
        print(f'**{col}**')
        le = df1[col].nunique()
        print ('Unique Values : ', df1[col].nunique())
        print(' ')
        if col in cat_cols and le < 13:
            print(df1[col].value_counts())
        if col in num_cols and  le < 23:
             print(df1[col].value_counts().sort_index(ascending=True)) 
        elif col in num_cols and le <150:
            print(df1[col].value_counts(bins=10, sort=False).sort_index(ascending=True))
        elif col in num_cols and le <1001:
            print(df1[col].value_counts(bins=100, sort=False).sort_index(ascending=True))

        print('=====================================================')

**parcelid**
Unique Values :  77381
 
**basementsqft**
Unique Values :  43
 
(34.477000000000004, 390.2]    20
(390.2, 742.4]                 16
(742.4, 1094.6]                 7
(1094.6, 1446.8]                3
(1446.8, 1799.0]                0
(1799.0, 2151.2]                2
(2151.2, 2503.4]                0
(2503.4, 2855.6]                0
(2855.6, 3207.8]                1
(3207.8, 3560.0]                1
Name: basementsqft, dtype: int64
**bathroomcnt**
Unique Values :  22
 
0.0       592
1.0     12902
1.5      1415
2.0     31485
2.5      6604
3.0     17313
3.5      1034
4.0      3352
4.5       695
5.0      1026
5.5       224
6.0       417
6.5        47
7.0       113
7.5        16
8.0       108
8.5         3
9.0        23
10.0        7
11.0        3
13.0        1
18.0        1
Name: bathroomcnt, dtype: int64
**bedroomcnt**
Unique Values :  16
 
0.0       830
1.0      3379
2.0     19169
3.0     30362
4.0     17513
5.0      4543
6.0       998
7.0       208
8.0       252
9.0      

In [21]:
def report_unique_val (df):
    num_cols = df.select_dtypes(exclude = 'O').columns.to_list()
    cat_cols = df.select_dtypes(include = 'O').columns.to_list()
    for col in df.columns:
            print(f'**{col}**')
            le = df[col].nunique()
            print ('Unique Values : ', df[col].nunique())
            print(' ')
            if col in cat_cols and le < 15:
                print(df[col].value_counts())
            if col in num_cols and  le < 23:
                 print(df[col].value_counts().sort_index(ascending=True)) 
            elif col in num_cols and le <150:
                print(df[col].value_counts(bins=10, sort=False).sort_index(ascending=True))
            elif col in num_cols and le <1001:
                print(df[col].value_counts(bins=100, sort=False).sort_index(ascending=True))

            print('=====================================================')

In [22]:
report_unique_val(df1)

**parcelid**
Unique Values :  77381
 
**basementsqft**
Unique Values :  43
 
(34.477000000000004, 390.2]    20
(390.2, 742.4]                 16
(742.4, 1094.6]                 7
(1094.6, 1446.8]                3
(1446.8, 1799.0]                0
(1799.0, 2151.2]                2
(2151.2, 2503.4]                0
(2503.4, 2855.6]                0
(2855.6, 3207.8]                1
(3207.8, 3560.0]                1
Name: basementsqft, dtype: int64
**bathroomcnt**
Unique Values :  22
 
0.0       592
1.0     12902
1.5      1415
2.0     31485
2.5      6604
3.0     17313
3.5      1034
4.0      3352
4.5       695
5.0      1026
5.5       224
6.0       417
6.5        47
7.0       113
7.5        16
8.0       108
8.5         3
9.0        23
10.0        7
11.0        3
13.0        1
18.0        1
Name: bathroomcnt, dtype: int64
**bedroomcnt**
Unique Values :  16
 
0.0       830
1.0      3379
2.0     19169
3.0     30362
4.0     17513
5.0      4543
6.0       998
7.0       208
8.0       252
9.0      

**3. Write a function that takes in a dataframe of observations and attributes and returns a dataframe where each row is an atttribute name, the first column is the number of rows with missing values for that attribute, and the second column is percent of total rows that have missing values for that attribute. Run the function and document takeaways from this on how you want to handle missing values.**

In [23]:
a.miss_dup_values(df1)

Your selected dataframe has 35 columns.
There are 25 columns that have missing values.
  
** There are 0 duplicate rows that represents 0.0% of total Values**


,Missing Values,% of Total Values
buildingclassdesc,77366,100.0
buildingclasstypeid,77366,100.0
basementsqft,77331,99.9
storydesc,77331,99.9
storytypeid,77331,99.9
architecturalstyledesc,77175,99.7
architecturalstyletypeid,77175,99.7
typeconstructiondesc,77159,99.7
typeconstructiontypeid,77159,99.7
decktypeid,76767,99.2


**4. Write a function that takes in a dataframe and returns a dataframe with 3 columns: the number of columns missing, percent of columns missing, and number of rows with n columns missing. Run the function and document takeaways from this on how you want to handle missing values.**

In [24]:
def nulls_by_row(df):
    '''
    akes in a dataframe and returns a dataframe with 3 columns: 
    the number of columns missing, percent of columns missing, and number of rows with n columns missing
    '''
    num_missing = df.isnull().sum(axis=1)
    prcnt_miss = num_missing / df.shape[1] * 100
    rows_missing = pd.DataFrame({'num_cols_missing': num_missing, 'percent_cols_missing': prcnt_miss})\
    .reset_index()\
    .groupby(['num_cols_missing', 'percent_cols_missing']).count()\
    .rename(index=str, columns={'index': 'num_rows'}).reset_index()
    return rows_missing

In [25]:
nulls_by_row (df1)

,num_cols_missing,percent_cols_missing,num_rows
0,8,22.857142857142858,1
1,9,25.71428571428571,24
2,10,28.57142857142857,75
3,11,31.428571428571427,56
4,12,34.285714285714285,188
5,13,37.142857142857146,250
6,14,40.0,7836
7,15,42.857142857142854,15061
8,16,45.714285714285715,7615
9,17,48.57142857142857,28234


## Prepare 

**1. Remove any properties that are likely to be something other than single unit properties. (e.g. no duplexes, no land/lot, ...). There are multiple ways to estimate that a property is a single unit, and there is not a single "right" answer. But for this exercise, do not purely filter by unitcnt as we did previously. Add some new logic that will reduce the number of properties that are falsely removed. You might want to use # bedrooms, square feet, unit type or the like to then identify those with unitcnt not defined.**

In [26]:
#lest see the columns that help me to identify single unit properties
df1[['parcelid', 'bedroomcnt', 'bathroomcnt', 'propertylandusedesc', 'calculatedfinishedsquarefeet','unitcnt']].head()


,parcelid,bedroomcnt,bathroomcnt,propertylandusedesc,calculatedfinishedsquarefeet,unitcnt
0,14297519,4.0,3.5,Single Family Residential,3100.0,NaN
1,17052889,2.0,1.0,Single Family Residential,1465.0,NaN
2,14186244,3.0,2.0,Single Family Residential,1243.0,NaN
3,12177905,4.0,3.0,Single Family Residential,2376.0,1.0
4,10887214,3.0,3.0,Condominium,1312.0,1.0


In [27]:
#check the values for propertylanduse
df1.propertylandusedesc.value_counts()

Single Family Residential                     52320
Condominium                                   19294
Duplex (2 Units, Any Combination)              2009
Planned Unit Development                       1944
Quadruplex (4 Units, Any Combination)           727
Triplex (3 Units, Any Combination)              535
Cluster Home                                    333
Mobile Home                                      74
Manufactured, Modular, Prefabricated Homes       58
Residential General                              37
Cooperative                                      29
Commercial/Office/Residential Mixed Used         15
Townhouse                                         6
Name: propertylandusedesc, dtype: int64

In [28]:
#this list has all types of single unit properties
single= ['Single Family Residential',' Mobile Home' , 'Townhouse '  ]


In [29]:
#use isin() to  get the all the properties of this type
df1['propertylandusedesc'].isin(single).head()

0     True
1     True
2     True
3     True
4    False
Name: propertylandusedesc, dtype: bool

In [30]:
#checking the condition
df1[['unitcnt']][(df1['unitcnt'] == 1) | (df1['unitcnt'] == 'NaN')].value_counts()

unitcnt
1.0        47293
dtype: int64

In [31]:
#checking if we get nulls
df1[['unitcnt']][(df1['unitcnt'] == 1) | (df1['unitcnt'].isnull())].isnull().sum()

unitcnt    26818
dtype: int64

In [32]:
#create a mask
single_mask = df1['propertylandusedesc'].isin(single)

In [33]:
#using that mask and also addind a condition
df1[single_mask | (df1['unitcnt'] == 1)].shape

(65879, 35)

In [34]:
#using that mask and also add  a condition
df_single = df1[single_mask & ((df1['unitcnt'] == 1) | (df1['unitcnt'].isnull()))]

In [35]:
df_single.head()

,parcelid,basementsqft,bathroomcnt,bedroomcnt,decktypeid,calculatedfinishedsquarefeet,fips,fireplacecnt,garagecarcnt,hashottuborspa,...,storydesc,propertylandusetypeid,propertylandusedesc,typeconstructiontypeid,typeconstructiondesc,unitcnt,taxvaluedollarcnt,taxamount,logerror,transactiondate
0,14297519,NaN,3.5,4.0,NaN,3100.0,6059.0,NaN,2.0,NaN,...,None,261.0,Single Family Residential,NaN,None,NaN,1023282.0,11013.72,0.025595,2017-01-01
1,17052889,NaN,1.0,2.0,NaN,1465.0,6111.0,1.0,1.0,NaN,...,None,261.0,Single Family Residential,NaN,None,NaN,464000.0,5672.48,0.055619,2017-01-01
2,14186244,NaN,2.0,3.0,NaN,1243.0,6059.0,NaN,2.0,NaN,...,None,261.0,Single Family Residential,NaN,None,NaN,564778.0,6488.30,0.005383,2017-01-01
3,12177905,NaN,3.0,4.0,NaN,2376.0,6037.0,NaN,NaN,NaN,...,None,261.0,Single Family Residential,NaN,None,1.0,145143.0,1777.51,-0.103410,2017-01-01
6,12095076,NaN,3.0,4.0,NaN,2962.0,6037.0,NaN,NaN,NaN,...,None,261.0,Single Family Residential,NaN,None,1.0,773303.0,9516.26,-0.001011,2017-01-01


In [36]:
df_single.shape

(52292, 35)

In [37]:
df_single['unitcnt'].value_counts()

1.0    33734
Name: unitcnt, dtype: int64

In [38]:
df_single['unitcnt'].isna().sum()

18558

In [39]:
df_single[['propertylandusedesc']][df_single['unitcnt'] > 1]

,propertylandusedesc


**2. Create a function that will drop rows or columns based on the percent of values that are missing: handle_missing_values(df, prop_required_column, prop_required_row).**
- The input:
    - A dataframe
        -A number between 0 and 1 that represents the proportion, for each column, of rows with non-missing values required to keep the column. i.e. if prop_required_column = .6, then you are requiring a column to have at least 60% of values not-NA (no more than 40% missing).
    - A number between 0 and 1 that represents the proportion, for each row, of columns/variables with non-missing values required to keep the row. For example, if prop_required_row = .75, then you are requiring a row to have at least 75% of variables with a non-missing value (no more that 25% missing).
- The output:
    - The dataframe with the columns and rows dropped as indicated. Be sure to drop the columns prior to the rows in your function.
- hint:
    - Look up the dropna documentation.
    - You will want to compute a threshold from your input values (prop_required) and total number of rows or columns.
    - Make use of inplace, i.e. inplace=True/False.


In [40]:
a.miss_dup_values(df_single)

Your selected dataframe has 35 columns.
There are 25 columns that have missing values.
  
** There are 0 duplicate rows that represents 0.0% of total Values**


,Missing Values,% of Total Values
buildingclassdesc,52292,100.0
buildingclasstypeid,52292,100.0
basementsqft,52245,99.9
storydesc,52245,99.9
storytypeid,52245,99.9
architecturalstyledesc,52222,99.9
architecturalstyletypeid,52222,99.9
typeconstructiondesc,52216,99.9
typeconstructiontypeid,52216,99.9
decktypeid,51904,99.3


In [41]:
nulls_by_row(df_single)

,num_cols_missing,percent_cols_missing,num_rows
0,8,22.857142857142858,1
1,9,25.71428571428571,24
2,10,28.57142857142857,8
3,11,31.428571428571427,40
4,12,34.285714285714285,150
5,13,37.142857142857146,228
6,14,40.0,3920
7,15,42.857142857142854,8699
8,16,45.714285714285715,5855
9,17,48.57142857142857,24395


In [42]:
a.miss_dup_values(df_single)

Your selected dataframe has 35 columns.
There are 25 columns that have missing values.
  
** There are 0 duplicate rows that represents 0.0% of total Values**


,Missing Values,% of Total Values
buildingclassdesc,52292,100.0
buildingclasstypeid,52292,100.0
basementsqft,52245,99.9
storydesc,52245,99.9
storytypeid,52245,99.9
architecturalstyledesc,52222,99.9
architecturalstyletypeid,52222,99.9
typeconstructiondesc,52216,99.9
typeconstructiontypeid,52216,99.9
decktypeid,51904,99.3


In [43]:
def handle_missing_values(df, prop_required_columns=0.5, prop_required_row=0.75):
    threshold = int(round(prop_required_columns * len(df.index),0))
    df = df.dropna(axis=1, thresh=threshold)
    threshold = int(round(prop_required_row * len(df.columns),0))
    df = df.dropna(axis=0, thresh=threshold)
    
    #drop rows with null values < 1
    lis =((100 * df.isnull().sum() / len(df))> 0) &  ((100 * df.isnull().sum() / len(df))< 1)
    col_drop = list(lis[lis == True].index)
    df = df.dropna(axis=0, subset = col_drop)
     
    return df

In [44]:
lis =((100 * df_single.isnull().sum() / len(df_single))> 0) &  ((100 * df_single.isnull().sum() / len(df_single))< 1)

In [45]:
col_drop = list(lis[lis == True].index)

In [46]:
col_drop

['calculatedfinishedsquarefeet',
 'lotsizesquarefeet',
 'yearbuilt',
 'taxvaluedollarcnt',
 'taxamount']

In [47]:
df_single.shape

(52292, 35)

In [48]:
df_clean= handle_missing_values(df_single, .5, .75)


In [49]:
df_clean.shape

(51816, 18)

**3. Decide how to handle the remaining missing values:**

- Fill with constant value.
- Impute with mean, median, mode.
- Drop row/column

**Note: handle_missing_values also remove the rows that has columuns with missing values less than 1%**

In [50]:
missing = a.miss_dup_values(df_clean)
missing

Your selected dataframe has 18 columns.
There are 3 columns that have missing values.
  
** There are 0 duplicate rows that represents 0.0% of total Values**


,Missing Values,% of Total Values
unitcnt,18400,35.5
heatingorsystemtypeid,18310,35.3
heatingorsystemdesc,18310,35.3


In [51]:
missing.index.to_list()

['unitcnt', 'heatingorsystemtypeid', 'heatingorsystemdesc']

In [52]:
#checking thecolumns that have missing values
df_clean[missing.index.to_list()].describe().T

,count,mean,std,min,25%,50%,75%,max
unitcnt,33416.0,1.000000,0.000000,1.0,1.0,1.0,1.0,1.0
heatingorsystemtypeid,33506.0,3.964066,2.531442,1.0,2.0,2.0,7.0,24.0


In [53]:
#for calculatedfinishedsquarefeet I will check if there any other house with the same # bedrooms and bathrooms
df_clean[df_clean['calculatedfinishedsquarefeet'].isnull()]

,parcelid,bathroomcnt,bedroomcnt,calculatedfinishedsquarefeet,fips,latitude,longitude,lotsizesquarefeet,yearbuilt,heatingorsystemtypeid,heatingorsystemdesc,propertylandusetypeid,propertylandusedesc,unitcnt,taxvaluedollarcnt,taxamount,logerror,transactiondate


In [54]:
#checking if there are another properties with same bedrooms, bathrooms and lotsize

In [55]:
df_clean[(df_clean['bathroomcnt'] == 4.5 ) & (df_clean['bedroomcnt'] == 5 ) & (df_clean['lotsizesquarefeet'] > 200000)]

,parcelid,bathroomcnt,bedroomcnt,calculatedfinishedsquarefeet,fips,latitude,longitude,lotsizesquarefeet,yearbuilt,heatingorsystemtypeid,heatingorsystemdesc,propertylandusetypeid,propertylandusedesc,unitcnt,taxvaluedollarcnt,taxamount,logerror,transactiondate
30668,17208766,4.5,5.0,5425.0,6111.0,34249871.0,-118867341.0,241758.0,2000.0,NaN,None,261.0,Single Family Residential,NaN,1756425.0,18729.14,-0.064421,2017-04-28


** I decide to drop those rows**

In [56]:
df_clean.dropna(axis=0, subset=['calculatedfinishedsquarefeet'], inplace = True)

In [57]:
df_clean[df_clean['calculatedfinishedsquarefeet'].isnull()]

,parcelid,bathroomcnt,bedroomcnt,calculatedfinishedsquarefeet,fips,latitude,longitude,lotsizesquarefeet,yearbuilt,heatingorsystemtypeid,heatingorsystemdesc,propertylandusetypeid,propertylandusedesc,unitcnt,taxvaluedollarcnt,taxamount,logerror,transactiondate


In [58]:
#for  unitcnt I will add 1 because it is single unit property

In [59]:
df_clean['unitcnt'].fillna(1, inplace= True)

In [60]:
df_clean['unitcnt'].isnull().sum()

0

In [61]:
a.miss_dup_values(df_clean)

Your selected dataframe has 18 columns.
There are 2 columns that have missing values.
  
** There are 0 duplicate rows that represents 0.0% of total Values**


,Missing Values,% of Total Values
heatingorsystemtypeid,18310,35.3
heatingorsystemdesc,18310,35.3


In [62]:
# lets check heatingorsystemtypeid and  heatingorsystemtypeid. they have the same information
df_clean['heatingorsystemdesc'].isnull()

0         True
1         True
2         True
3        False
6        False
         ...  
77388    False
77575    False
77577    False
77576     True
77578    False
Name: heatingorsystemdesc, Length: 51816, dtype: bool

In [63]:
df_clean['heatingorsystemdesc'].value_counts()

Central       20430
Floor/Wall    12471
Forced air      512
Solar            63
None             16
Baseboard         7
Radiant           4
Gravity           2
Yes               1
Name: heatingorsystemdesc, dtype: int64

In [64]:
df_clean['heatingorsystemtypeid'].value_counts()

2.0     20430
7.0     12471
6.0       512
20.0       63
13.0       16
1.0         7
18.0        4
10.0        2
24.0        1
Name: heatingorsystemtypeid, dtype: int64

In [65]:
#I will add with the most frequent value

In [66]:
df_clean= df_clean.drop(columns = 'heatingorsystemdesc')

In [67]:
most_f =df_clean['heatingorsystemtypeid'].mode()[0]
most_f

2.0

In [68]:
df_clean['heatingorsystemtypeid'].fillna(most_f, inplace= True)

In [69]:
df_clean['heatingorsystemtypeid'].isnull().sum()

0

In [70]:
#let's see the columns with missing values

In [71]:
miss = a.miss_dup_values(df_clean)
miss

Your selected dataframe has 17 columns.
There are 0 columns that have missing values.
  
** There are 0 duplicate rows that represents 0.0% of total Values**


,Missing Values,% of Total Values


In [72]:
df_clean['yearbuilt'].describe()

count    51816.000000
mean      1963.287537
std         22.961239
min       1878.000000
25%       1950.000000
50%       1960.000000
75%       1979.000000
max       2016.000000
Name: yearbuilt, dtype: float64

In [73]:
#I will drop them 
list_miss = miss.index.to_list()

In [74]:
#df_clean.drop(columns = list_miss)

In [75]:
a.miss_dup_values(df_clean)

Your selected dataframe has 17 columns.
There are 0 columns that have missing values.
  
** There are 0 duplicate rows that represents 0.0% of total Values**


,Missing Values,% of Total Values


In [76]:
df_clean.shape

(51816, 17)

In [77]:
df_clean= df_clean.dropna(axis = 0, subset = list_miss )

In [78]:
df_clean.shape

(51816, 17)

In [79]:
a.miss_dup_values(df_clean)

Your selected dataframe has 17 columns.
There are 0 columns that have missing values.
  
** There are 0 duplicate rows that represents 0.0% of total Values**


,Missing Values,% of Total Values


### wrangle_zillow.py
**Functions of the work above needed to acquire and prepare a new sample of data.**

In [80]:
def wrangle_zillow ( sql_query, prop_required_columns=0.5, prop_required_row=0.75):
    
    #acquire data
    df= a.get_data_from_sql('zillow',sql_query)
    
    #getting the latest transactions 
    df1 = df.sort_values(by ='transactiondate', ascending=True).drop_duplicates( subset = 'parcelid' ,keep= 'last')
    
    #this list has all types of single unit properties
    single= ['Single Family Residential',' Mobile Home' , 'Townhouse '  ]
    #create a mask
    single_mask = df1['propertylandusedesc'].isin(single)
    #using that mask and also add  a condition
    df_single = df1[single_mask & ((df1['unitcnt'] == 1) | (df1['unitcnt'].isnull()))]
    
    #missing values
    df_clean = handle_missing_values(df_single, prop_required_columns, prop_required_row)
    
    #fill missing values in unitcnt
    df_clean['unitcnt'].fillna(1, inplace= True)
    
    #fill missing values heatingorsystemtypeid
    most_f =df_clean['heatingorsystemtypeid'].mode()[0]
    df_clean['heatingorsystemtypeid'].fillna(most_f, inplace= True)
    
    #drop duplicated rows
    df_clean= df_clean.drop(columns = 'heatingorsystemdesc')
    
    return df_clean
    

In [81]:
df =wrangle_zillow ( sql_query, prop_required_columns=0.5, prop_required_row=0.75)

In [82]:
df.head()

,parcelid,bathroomcnt,bedroomcnt,calculatedfinishedsquarefeet,fips,latitude,longitude,lotsizesquarefeet,yearbuilt,heatingorsystemtypeid,propertylandusetypeid,propertylandusedesc,unitcnt,taxvaluedollarcnt,taxamount,logerror,transactiondate
0,14297519,3.5,4.0,3100.0,6059.0,33634931.0,-117869207.0,4506.0,1998.0,2.0,261.0,Single Family Residential,1.0,1023282.0,11013.72,0.025595,2017-01-01
1,17052889,1.0,2.0,1465.0,6111.0,34449266.0,-119281531.0,12647.0,1967.0,2.0,261.0,Single Family Residential,1.0,464000.0,5672.48,0.055619,2017-01-01
2,14186244,2.0,3.0,1243.0,6059.0,33886168.0,-117823170.0,8432.0,1962.0,2.0,261.0,Single Family Residential,1.0,564778.0,6488.30,0.005383,2017-01-01
3,12177905,3.0,4.0,2376.0,6037.0,34245180.0,-118240722.0,13038.0,1970.0,2.0,261.0,Single Family Residential,1.0,145143.0,1777.51,-0.103410,2017-01-01
6,12095076,3.0,4.0,2962.0,6037.0,34145202.0,-118179824.0,63000.0,1950.0,2.0,261.0,Single Family Residential,1.0,773303.0,9516.26,-0.001011,2017-01-01


In [83]:
df.shape

(51816, 17)

In [84]:
a.miss_dup_values(df)

Your selected dataframe has 17 columns.
There are 0 columns that have missing values.
  
** There are 0 duplicate rows that represents 0.0% of total Values**


,Missing Values,% of Total Values


In [85]:
import wrangle_zillow as w

In [86]:
df = w.wrangle_zillow(sql_query2,prop_required_columns=0.5,prop_required_row=0.75,)

In [87]:
df.head()

,parcelid,bathroomcnt,bedroomcnt,calculatedfinishedsquarefeet,fips,latitude,longitude,lotsizesquarefeet,yearbuilt,heatingorsystemtypeid,propertylandusetypeid,propertylandusedesc,unitcnt,taxvaluedollarcnt,taxamount,logerror,transactiondate
0,14297519,3.5,4.0,3100.0,6059.0,33634931.0,-117869207.0,4506.0,1998.0,2.0,261.0,Single Family Residential,1.0,1023282.0,11013.72,0.025595,2017-01-01
1,17052889,1.0,2.0,1465.0,6111.0,34449266.0,-119281531.0,12647.0,1967.0,2.0,261.0,Single Family Residential,1.0,464000.0,5672.48,0.055619,2017-01-01
2,14186244,2.0,3.0,1243.0,6059.0,33886168.0,-117823170.0,8432.0,1962.0,2.0,261.0,Single Family Residential,1.0,564778.0,6488.30,0.005383,2017-01-01
3,12177905,3.0,4.0,2376.0,6037.0,34245180.0,-118240722.0,13038.0,1970.0,2.0,261.0,Single Family Residential,1.0,145143.0,1777.51,-0.103410,2017-01-01
6,12095076,3.0,4.0,2962.0,6037.0,34145202.0,-118179824.0,63000.0,1950.0,2.0,261.0,Single Family Residential,1.0,773303.0,9516.26,-0.001011,2017-01-01


In [88]:
df.shape

(51816, 17)

In [89]:
a.miss_dup_values(df)

Your selected dataframe has 17 columns.
There are 0 columns that have missing values.
  
** There are 0 duplicate rows that represents 0.0% of total Values**


,Missing Values,% of Total Values


In [90]:
a.summarize(df)

Dataframe shape: 
(51816, 17)
Dataframe head: 
   parcelid  bathroomcnt  bedroomcnt  calculatedfinishedsquarefeet    fips  \
0  14297519          3.5         4.0                        3100.0  6059.0   
1  17052889          1.0         2.0                        1465.0  6111.0   
2  14186244          2.0         3.0                        1243.0  6059.0   

     latitude    longitude  lotsizesquarefeet  yearbuilt  \
0  33634931.0 -117869207.0             4506.0     1998.0   
1  34449266.0 -119281531.0            12647.0     1967.0   
2  33886168.0 -117823170.0             8432.0     1962.0   

   heatingorsystemtypeid  propertylandusetypeid        propertylandusedesc  \
0                    2.0                  261.0  Single Family Residential   
1                    2.0                  261.0  Single Family Residential   
2                    2.0                  261.0  Single Family Residential   

   unitcnt  taxvaluedollarcnt  taxamount  logerror transactiondate  
0      1.0       

Unique Values :  1
 
Single Family Residential    51816
Name: propertylandusedesc, dtype: int64
**unitcnt**
Unique Values :  1
 
1.0    51816
Name: unitcnt, dtype: int64
**taxvaluedollarcnt**
Unique Values :  38597
 
**taxamount**
Unique Values :  50359
 
**logerror**
Unique Values :  51561
 
**transactiondate**
Unique Values :  257
 
